# Pythonにおけるデータタイプの理解

## PythonのIntegerはただのInteger以上のもの
Python 3.4のソースコードを見ると、integer (long)は次の通り定義されているようである。
```c
struct _longobject {
    long ob_refcnt;
    PyTypeObject *ob_type;
    size_t ob_size;
    long ob_digit[1];
};
```
Python 3.4のIntegerは次の4つの要素からなる。つまり、C言語で単純に扱う場合よりもオーバーヘッドが存在することを意味する。

- `ob_refcnt`: メモリの確保、開放を助けるレファレンスカウント
- `ob_type`: 変数の型
- `ob_size`: データメンバーのサイズ
- `ob_digit`: 実際のintegerの値

## PythonのListはただのList以上のもの

In [1]:
# Integerのリスト
L = list(range(10))
L

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [2]:
type(L[0])

int

In [3]:
# stringのリスト
L2 = [str(c) for c in L]
L2

['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']

In [4]:
type(L2[0])

str

In [5]:
# それぞれ型の違うリスト
L3 = [True, "2", 3.0, 4]
[type(item) for item in L3]

[bool, str, float, int]

上記のような柔軟さは処理コストがかかる。

これらの柔軟なタイプを可能とするためには、リスト内のそれぞれのアイテムが自分自身の型情報や、リファレンスカウント、そして他の情報などを含む必要がある。そして、それぞれのアイテムが完全なPythonオブジェクトである。

すべての変数が同じ型であるようなケースに置いては、これらの情報は冗長である。

numpyのarrayのように型を固定したarrayでデータをストアすることで、より効率的になる。

実装レベルでは、arrayは一つの連続したデータブロックに対する単独のポインターを持っている。一方、Pythonのリストはポインターのブロックに対する一つのポインターを持つ。それらのポインターはPython integerのような完全なPythonオブジェクトに対するポインターである。

## 型が固定されたArray
Python 3.3からはビルトインで`array`モジュールが利用できる。

In [6]:
import array
L = list(range(10))
A = array.array('i', L)
A

array('i', [0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

ここで`'i'`は中身がintegersであることを示す。

しかし、より便利なのはNumPyパッケージの`ndarray`オブジェクトである。

Pythonの`array`オブジェクトがarrayベースのデータをより効率的に保存できる一方で、NumPyはそれらのデータに対して効率的なオペレーションを加える。

それでは、NumPyを`np`のエイリアスのもとでimportすることから始める。

In [7]:
import numpy as np

## PythonリストからArrayを作成する
始めに、Pythonリストからarrayを作成するのに`np.array`を利用することができる。

In [8]:
# ineger array:
np.array([1, 4, 2, 5, 3])

array([1, 4, 2, 5, 3])

Pythonリストと違って、NumPyはすべて同じ型のデータで構成されなければならないという制限があることを思い出そう。もし型が違ったなら、NumPyは可能ならアップキャストする。（ここではintegerがfloating pointにup-castされている）

In [9]:
np.array([3.14, 4, 2, 3])

array([3.14, 4.  , 2.  , 3.  ])

もし、得られるarrayのデータ型を厳密に設定したいならば、`dtype`キーワードを使用する事ができる。

In [10]:
np.array([1, 2, 3, 4], dtype='float32')

array([1., 2., 3., 4.], dtype=float32)

最後に、Pythonのリストとは違い、NumPyのarrayは厳密に多次元とすることができる。
以下は、リストのリストを利用した多次元arrayの初期化方法の一つである。

In [11]:
np.array([range(i, i+3) for i in [2, 4, 6]])

array([[2, 3, 4],
       [4, 5, 6],
       [6, 7, 8]])

## スクラッチからのarrayの作成
特に大きなarrayに対して、NumPyで用意されたルーチンを使用してスクラッチでarrayを作成することはより効率的である。以下はいくつかの例である。

In [12]:
# Create a length-10 integer array filled with zeros
np.zeros(10, dtype=int)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [13]:
# Create a 3x5 floating-point array filled with ones
np.ones((3, 5), dtype=float)

array([[1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.]])

In [14]:
# Create a 3x5 array filled with 3.14
np.full((3, 5), 3.14)

array([[3.14, 3.14, 3.14, 3.14, 3.14],
       [3.14, 3.14, 3.14, 3.14, 3.14],
       [3.14, 3.14, 3.14, 3.14, 3.14]])

In [15]:
# Create an array filled with a linear sequence
# Starting at 0, ending at 20, stepping by 2
# (this is similar to the built-in range() function)
np.arange(0, 20, 2)

array([ 0,  2,  4,  6,  8, 10, 12, 14, 16, 18])

In [16]:
# Create an array of five values evenly spaced between 0 and 1
np.linspace(0, 1, 5)

array([0.  , 0.25, 0.5 , 0.75, 1.  ])

In [17]:
# Create a 3x3 array of uniformly distributed
# random values between 0 and 1
np.random.random((3, 3))

array([[0.2964048 , 0.27615584, 0.97248679],
       [0.26626313, 0.38452642, 0.30534207],
       [0.51729955, 0.42938415, 0.98669105]])

In [18]:
# Create a 3x3 array of normally distributed random values
# with mean 0 and standard deviation 1
np.random.normal(0, 1, (3, 3))

array([[ 0.73846423,  0.66925306,  0.40249114],
       [-0.05239892,  0.59851037, -0.43106587],
       [-0.51343619, -0.75711411,  0.34091276]])

In [19]:
# Create a 3x3 array of random integers in the interval [0, 10)
np.random.randint(0, 10, (3, 3))

array([[8, 5, 1],
       [2, 8, 2],
       [9, 2, 6]])

In [20]:
# Create a 3x3 identity matrix
np.eye(3)

array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]])

In [21]:
# Create an uninitialized array of three integers
# The values will be whatever happens to already exist at that memory location
np.empty(3)

array([1., 1., 1.])

## NumPyの標準的なデータタイプ
NumPyのarrayは一つの型の値で成り立つため、それらの型と限界の詳細な知識を持つことは重要である。NumPyはCでビルドされているため、CやFortlan、その他の関連する言語の使用者にはその型は馴染みやすいであろう。

標準的なNumPyの型は以下のテーブルに記されるとおりである。arrayを構成する時は、それらはstringを使用して指定される。

In [22]:
np.zeros(10, dtype='int16')

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int16)

もしくは、関連するNumPyオブジェクトを使用する。

In [23]:
np.zeros(10, dtype=np.int16)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int16)

|Data type|Description|
|:--------|:----------|
|bool_|Boolean (True or False) stored as a byte|
|int_|Default integer type (same as C long; normally either int64 or int32)|
|intc|Identical to C int (normally int32 or int64)|
|intp|Integer used for indexing (same as C ssize_t; normally either int32 or int64)|
|int8|Byte (-128 to 127)|
|int16|Integer (-32768 to 32767)|
|int32|Integer (-2147483648 to 2147483647)|
|int64|Integer (-9223372036854775808 to 9223372036854775807)|
|uint8|Unsigned integer (0 to 255)|
|uint16|Unsigned integer (0 to 65535)|
|uint32|Unsigned integer (0 to 4294967295)|
|uint64|Unsigned integer (0 to 18446744073709551615)|
|float_|Shorthand for float64.|
|float16|Half precision float: sign bit, 5 bits exponent, 10 bits mantissa|
|float32|Single precision float: sign bit, 8 bits exponent, 23 bits mantissa|
|float64|Double precision float: sign bit, 11 bits exponent, 52 bits mantissa|
|complex_|Shorthand for complex128.|
|complex64|Complex number, represented by two 32-bit floats|
|complex128|Complex number, represented by two 64-bit floats|

より応用的な型の指定も可能である。詳細は[NumPy documentation](http://numpy.org/)を参照。